# Task 2: Quantitative Analysis using PyNance and TA-Lib

This notebook performs comprehensive quantitative analysis on stock price data using technical indicators and financial metrics.

## Analysis Components:
1. **Data Loading**: Load stock price data using yfinance
2. **Technical Indicators (TA-Lib)**: Moving Averages, RSI, MACD, Bollinger Bands
3. **Financial Metrics (PyNance)**: Returns, Volatility, Sharpe Ratio, Beta, Alpha, Drawdown
4. **Visualization**: Comprehensive charts and analysis
5. **KPI Analysis**: Key Performance Indicators and metrics


## 1. Setup and Imports


In [ ]:
import sys
import os
sys.path.append('../scripts')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Import custom modules
from load_stock_data import (
    load_stock_data,
    load_multiple_stocks,
    prepare_stock_data,
    validate_stock_data
)
from technical_indicators import (
    calculate_moving_averages,
    calculate_rsi,
    calculate_macd,
    calculate_bollinger_bands,
    calculate_all_indicators,
    get_indicator_summary
)
from pynance_metrics import (
    calculate_returns,
    calculate_volatility,
    calculate_sharpe_ratio,
    calculate_max_drawdown,
    calculate_beta,
    calculate_alpha,
    calculate_all_metrics,
    get_metrics_summary
)
from visualize_indicators import (
    plot_price_with_indicators,
    plot_returns_analysis,
    plot_risk_metrics,
    plot_indicator_comparison,
    plot_correlation_heatmap
)

print("All modules imported successfully!")


## 2. Load Stock Price Data

We'll load stock price data for analysis. You can modify the ticker symbols and date ranges as needed.


In [ ]:
# Configuration
TICKER = 'AAPL'  # Change this to your desired stock ticker
START_DATE = '2020-01-01'  # Start date for data
END_DATE = None  # End date (None = today)
PERIOD = '2y'  # Alternative: use period instead of dates

# Load stock data
print(f"Loading data for {TICKER}...")
stock_df = load_stock_data(
    ticker=TICKER,
    start_date=START_DATE,
    end_date=END_DATE,
    period=PERIOD
)

# Prepare the data
stock_df = prepare_stock_data(stock_df)

print(f"\nData loaded successfully!")
print(f"Shape: {stock_df.shape}")
print(f"\nDate range: {stock_df['date'].min()} to {stock_df['date'].max()}")
print(f"\nColumns: {stock_df.columns.tolist()}")
print(f"\nFirst few rows:")
display(stock_df.head())

# Validate data
validation_report = validate_stock_data(stock_df)
print("\nData Validation Report:")
print("=" * 50)
for key, value in validation_report.items():
    print(f"{key}: {value}")


## 3. Calculate Technical Indicators using TA-Lib

We'll calculate various technical indicators including Moving Averages, RSI, MACD, and Bollinger Bands.


In [ ]:
# Calculate all technical indicators
print("Calculating technical indicators...")
stock_df = calculate_all_indicators(
    stock_df,
    include_moving_averages=True,
    include_rsi=True,
    include_macd=True,
    include_bollinger=True,
    include_stochastic=False
)

print("Technical indicators calculated successfully!")
print(f"\nNew columns added: {[col for col in stock_df.columns if col not in ['Open', 'High', 'Low', 'Close', 'Volume', 'date', 'ticker']]}")

# Display indicator summary
indicator_summary = get_indicator_summary(stock_df)
print("\nIndicator Summary:")
print("=" * 50)
for indicator, stats in indicator_summary.items():
    print(f"\n{indicator}:")
    for key, value in stats.items():
        if value is not None:
            print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")


## 4. Calculate Financial Metrics using PyNance

We'll calculate financial metrics including returns, volatility, Sharpe ratio, and other risk metrics.


In [ ]:
# Load market data for beta and alpha calculation (using S&P 500 as benchmark)
print("Loading market benchmark data (S&P 500)...")
try:
    market_df = load_stock_data(ticker='^GSPC', start_date=START_DATE, end_date=END_DATE, period=PERIOD)
    market_df = prepare_stock_data(market_df)
    market_returns = market_df['returns'] if 'returns' in market_df.columns else None
    
    if market_returns is None:
        market_df = calculate_returns(market_df)
        market_returns = market_df['returns']
    
    print("Market data loaded successfully!")
except Exception as e:
    print(f"Warning: Could not load market data: {e}")
    print("Beta and Alpha calculations will be skipped.")
    market_returns = None

# Calculate all financial metrics
print("\nCalculating financial metrics...")
stock_df = calculate_all_metrics(
    stock_df,
    market_returns=market_returns,
    risk_free_rate=0.02  # 2% annual risk-free rate
)

print("Financial metrics calculated successfully!")

# Display metrics summary
metrics_summary = get_metrics_summary(stock_df)
print("\nFinancial Metrics Summary:")
print("=" * 50)
for metric, stats in metrics_summary.items():
    print(f"\n{metric}:")
    for key, value in stats.items():
        if value is not None:
            if isinstance(value, float):
                if 'return' in key.lower() or 'alpha' in key.lower():
                    print(f"  {key}: {value*100:.2f}%")
                else:
                    print(f"  {key}: {value:.4f}")
            else:
                print(f"  {key}: {value}")


## 5. Visualizations

Create comprehensive visualizations to understand the data and indicators.


In [ ]:
# Set date as index for plotting
plot_df = stock_df.set_index('date') if 'date' in stock_df.columns else stock_df

# Plot 1: Price with Technical Indicators
plot_price_with_indicators(
    plot_df,
    price_column='Close',
    title=f"{TICKER} - Price with Technical Indicators",
    save_path='../outputs/price_with_indicators.png'
)


In [ ]:
# Plot 2: Returns Analysis
plot_returns_analysis(
    plot_df,
    returns_column='returns',
    title=f"{TICKER} - Returns Analysis",
    save_path='../outputs/returns_analysis.png'
)


In [ ]:
# Plot 3: Risk Metrics
plot_risk_metrics(
    plot_df,
    title=f"{TICKER} - Risk Metrics Analysis",
    save_path='../outputs/risk_metrics.png'
)


In [ ]:
# Plot 4: Indicator Correlation
indicator_columns = [
    'Close', 'SMA_20', 'SMA_50', 'RSI_14', 'MACD', 'MACD_signal',
    'BB_upper_20', 'BB_middle_20', 'BB_lower_20'
]
available_indicators = [col for col in indicator_columns if col in plot_df.columns]

plot_correlation_heatmap(
    plot_df,
    columns=available_indicators,
    title=f"{TICKER} - Indicator Correlation Matrix",
    save_path='../outputs/indicator_correlation.png'
)


## 6. Key Performance Indicators (KPIs)

Calculate and display key performance indicators for the analysis.


In [ ]:
# Calculate KPIs
print("Key Performance Indicators (KPIs)")
print("=" * 70)

# 1. Total Return
if 'returns' in stock_df.columns:
    total_return = (1 + stock_df['returns'].dropna()).prod() - 1
    annualized_return = (1 + stock_df['returns'].dropna().mean()) ** 252 - 1
    print(f"\n1. Returns:")
    print(f"   Total Return: {total_return*100:.2f}%")
    print(f"   Annualized Return: {annualized_return*100:.2f}%")

# 2. Volatility
vol_cols = [col for col in stock_df.columns if 'volatility' in col and 'annualized' in col]
if vol_cols:
    current_vol = stock_df[vol_cols[0]].dropna().iloc[-1] if len(stock_df[vol_cols[0]].dropna()) > 0 else None
    avg_vol = stock_df[vol_cols[0]].dropna().mean()
    print(f"\n2. Volatility:")
    print(f"   Current Annualized Volatility: {current_vol*100:.2f}%" if current_vol else "   N/A")
    print(f"   Average Annualized Volatility: {avg_vol*100:.2f}%")

# 3. Sharpe Ratio
sharpe_cols = [col for col in stock_df.columns if 'sharpe_ratio' in col]
if sharpe_cols:
    current_sharpe = stock_df[sharpe_cols[0]].dropna().iloc[-1] if len(stock_df[sharpe_cols[0]].dropna()) > 0 else None
    avg_sharpe = stock_df[sharpe_cols[0]].dropna().mean()
    print(f"\n3. Sharpe Ratio:")
    print(f"   Current Sharpe Ratio: {current_sharpe:.4f}" if current_sharpe else "   N/A")
    print(f"   Average Sharpe Ratio: {avg_sharpe:.4f}")

# 4. Maximum Drawdown
if 'max_drawdown_pct' in stock_df.columns:
    worst_drawdown = stock_df['max_drawdown_pct'].min()
    current_drawdown = stock_df['max_drawdown_pct'].iloc[-1]
    print(f"\n4. Maximum Drawdown:")
    print(f"   Worst Drawdown: {worst_drawdown:.2f}%")
    print(f"   Current Drawdown: {current_drawdown:.2f}%")

# 5. RSI Status
rsi_cols = [col for col in stock_df.columns if col.startswith('RSI_') and not '_' in col[4:]]
if rsi_cols:
    current_rsi = stock_df[rsi_cols[0]].iloc[-1]
    rsi_status = "Overbought" if current_rsi > 70 else "Oversold" if current_rsi < 30 else "Neutral"
    print(f"\n5. RSI Status:")
    print(f"   Current RSI: {current_rsi:.2f}")
    print(f"   Status: {rsi_status}")

# 6. MACD Signal
if 'MACD' in stock_df.columns and 'MACD_signal' in stock_df.columns:
    current_macd = stock_df['MACD'].iloc[-1]
    current_signal = stock_df['MACD_signal'].iloc[-1]
    macd_signal = "Bullish" if current_macd > current_signal else "Bearish"
    print(f"\n6. MACD Signal:")
    print(f"   MACD: {current_macd:.4f}")
    print(f"   Signal: {current_signal:.4f}")
    print(f"   Trend: {macd_signal}")

# 7. Beta (if available)
beta_cols = [col for col in stock_df.columns if 'beta' in col]
if beta_cols:
    current_beta = stock_df[beta_cols[0]].dropna().iloc[-1] if len(stock_df[beta_cols[0]].dropna()) > 0 else None
    avg_beta = stock_df[beta_cols[0]].dropna().mean()
    print(f"\n7. Beta (Market Sensitivity):")
    print(f"   Current Beta: {current_beta:.4f}" if current_beta else "   N/A")
    print(f"   Average Beta: {avg_beta:.4f}")
    if current_beta:
        beta_interpretation = "More volatile than market" if current_beta > 1 else "Less volatile than market" if current_beta < 1 else "Same as market"
        print(f"   Interpretation: {beta_interpretation}")

# 8. Alpha (if available)
alpha_cols = [col for col in stock_df.columns if 'alpha' in col]
if alpha_cols:
    current_alpha = stock_df[alpha_cols[0]].dropna().iloc[-1] if len(stock_df[alpha_cols[0]].dropna()) > 0 else None
    avg_alpha = stock_df[alpha_cols[0]].dropna().mean()
    print(f"\n8. Alpha (Excess Return):")
    print(f"   Current Alpha: {current_alpha*100:.2f}%" if current_alpha else "   N/A")
    print(f"   Average Alpha: {avg_alpha*100:.2f}%")

print("\n" + "=" * 70)


In [ ]:
print("Analysis Summary and Insights")
print("=" * 70)

# Data completeness
print(f"\n1. Data Completeness:")
print(f"   - Total data points: {len(stock_df)}")
print(f"   - Date range: {stock_df['date'].min()} to {stock_df['date'].max()}")
print(f"   - Missing values: {stock_df[['Open', 'High', 'Low', 'Close', 'Volume']].isnull().sum().sum()}")

# Indicator accuracy
print(f"\n2. Indicator Accuracy:")
indicators_calculated = len([col for col in stock_df.columns if any(x in col for x in ['SMA', 'EMA', 'RSI', 'MACD', 'BB'])])
print(f"   - Technical indicators calculated: {indicators_calculated}")
metrics_calculated = len([col for col in stock_df.columns if any(x in col for x in ['returns', 'volatility', 'sharpe', 'beta', 'alpha', 'drawdown'])])
print(f"   - Financial metrics calculated: {metrics_calculated}")

# Key findings
print(f"\n3. Key Findings:")
if 'returns' in stock_df.columns:
    positive_days = (stock_df['returns'].dropna() > 0).sum()
    total_days = len(stock_df['returns'].dropna())
    win_rate = (positive_days / total_days * 100) if total_days > 0 else 0
    print(f"   - Win rate: {win_rate:.2f}% ({positive_days}/{total_days} positive days)")

if 'RSI_14' in stock_df.columns:
    current_rsi = stock_df['RSI_14'].iloc[-1]
    if current_rsi > 70:
        print(f"   - RSI indicates overbought conditions (RSI: {current_rsi:.2f})")
    elif current_rsi < 30:
        print(f"   - RSI indicates oversold conditions (RSI: {current_rsi:.2f})")
    else:
        print(f"   - RSI indicates neutral conditions (RSI: {current_rsi:.2f})")

if 'MACD' in stock_df.columns:
    if stock_df['MACD'].iloc[-1] > stock_df['MACD_signal'].iloc[-1]:
        print(f"   - MACD shows bullish momentum")
    else:
        print(f"   - MACD shows bearish momentum")

print("\n" + "=" * 70)
print("\nAnalysis complete! Review the visualizations and KPIs above for insights.")
